[![imagenes](imagenes/pythonista.png)](https://pythonista.io)

# Teoría de Concurrencia, Paralelismo y Asincronía

Este cuaderno no contiene ejercicios de código para resolver, sino que sirve como una **fundamentación teórica exhaustiva** antes de sumergirnos en los módulos de `threading`, `multiprocessing` y `asyncio`.

## 1. Conceptos Fundamentales

A menudo usamos "concurrencia" y "paralelismo" de forma intercambiable, pero en ciencias de la computación son conceptos distintos.

### Concurrencia (Concurrency)
La capacidad de un sistema para **gestionar** múltiples tareas al mismo tiempo. No implica necesariamente que se estén ejecutando en el mismo instante físico, sino que sus ejecuciones se solapan en un periodo de tiempo. 
*   **Analogía**: Un chef picando cebolla, poniendo agua a hervir y vigilando el horno. Hace todo "a la vez", pero solo una cosa en cada instante preciso.

### Paralelismo (Parallelism)
La capacidad de **ejecutar** múltiples tareas simultáneamente en el mismo instante físico. Requiere hardware con múltiples núcleos de procesamiento.
*   **Analogía**: Tres chefs en la cocina; uno pica cebolla, otro vigila el agua y otro el horno. Todos trabajan en el mismo segundo exacto.

In [ ]:
import time
import threading
import multiprocessing

def tarea_simulada(nombre, duracion):
    print(f"Iniciando {nombre}...")
    time.sleep(duracion)
    print(f"Finalizando {nombre}...")

# Esto es ejecución secuencial (ni concurrente ni paralela)
start = time.time()
tarea_simulada("Tarea A", 1)
tarea_simulada("Tarea B", 1)
print(f"Tiempo secuencial: {time.time() - start:.2f}s")

## 2. El Runtime de Python y el GIL

CPython, la implementación estándar de Python, tiene un mecanismo llamado **Global Interpreter Lock (GIL)**. 

### ¿Qué es el GIL?
Es un semáforo (mutex) que protege el acceso a los objetos de Python, impidiendo que múltiples hilos nativos ejecuten bytecodes de Python simultáneamente en el mismo proceso. 

### Implicaciones
Incluso si tienes un procesador de 64 núcleos y lanzas 64 hilos en Python, **solo uno** podrá ejecutar código Python a la vez. El sistema operativo puede cambiar entre hilos (context switching), dando la ilusión de paralelismo, pero es **concurrencia**.

**Excepción importante**: Muchas operaciones de E/S (lectura de archivos, red) y ciertas extensiones en C (como NumPy) liberan el GIL, permitiendo verdadero paralelismo en esas operaciones específicas.

> **Novedad en Python 3.13+ (Experimental)**: A partir de la versión 3.13, se ha introducido soporte experimental para ejecutar Python sin el GIL (PEP 703). Esto permite que los hilos de Python se ejecuten en verdadero paralelismo en múltiples núcleos. Sin embargo, esto requiere utilizar una versión del intérprete compilada específicamente en modo "free-threaded" y es posible que algunas bibliotecas externas aún no sean compatibles.

In [ ]:
def cuenta_regresiva(n):
    while n > 0:
        n -= 1

COUNT = 50000000

# Prueba secuencial
start = time.time()
cuenta_regresiva(COUNT)
print(f"Secuencial: {time.time() - start:.4f}s")

# Prueba con Hilos (Threading) - CPU Bound
# Debido al GIL, esto NO será más rápido (y a menudo es más lento por el overhead)
t1 = threading.Thread(target=cuenta_regresiva, args=(COUNT // 2,))
t2 = threading.Thread(target=cuenta_regresiva, args=(COUNT // 2,))

start = time.time()
t1.start(); t2.start()
t1.join(); t2.join()
print(f"Con Hilos (CPU-Bound): {time.time() - start:.4f}s (¡Observa el impacto del GIL!)")

## 3. Tipos de Carga: CPU-Bound vs I/O-Bound

Para elegir la herramienta correcta, primero debes identificar qué limita tu programa.

### I/O-Bound (Limitado por Entrada/Salida)
El programa pasa la mayor parte del tiempo esperando: esperando respuesta de red, esperando escritura en disco, esperando input del usuario.
*   **Ejemplos**: Web scraping, servidores web, lectura de CSVs gigantes.
*   **Solución en Python**: `threading` o `asyncio`. El GIL no molesta aquí porque se libera durante la espera.

### CPU-Bound (Limitado por Procesador)
El programa pasa la mayor parte del tiempo calculando números.
*   **Ejemplos**: Procesamiento de imágenes, entrenamiento de modelos ML, compresión de video.
*   **Solución en Python**: `multiprocessing`. Necesitas procesos separados (cada uno con su propio GIL e intérprete) para usar múltiples núcleos.

## 4. Modelos de Ejecución Comparados

| Característica | Procesos (`multiprocessing`) | Hilos (`threading`) | Asincronía (`asyncio`) |
| :--- | :--- | :--- | :--- |
| **Memoria** | Aislada (cada proceso tiene su copia) | Compartida (todos acceden a lo mismo) | Compartida (un solo proceso/hilo) |
| **Overhead** | Alto (crear un proceso es costoso) | Medio (context switching del OS) | Muy Bajo (cambio de tarea en userspace) |
| **Paralelismo** | Sí (Multi-Core Real) | No (Concurrencia por GIL) | No (Concurrencia en 1 hilo) |
| **Uso Ideal** | Tareas CPU-Bound pesadas | Tareas I/O-Bound simples/legacy | Tareas I/O-Bound masivas (10k+ conexiones) |
| **Dificultad** | Media (serialización de datos) | Alta (Race conditions, Deadlocks) | Alta (Nuevo paradigma de programación) |

## 5. Leyes Teóricas del Rendimiento

### Ley de Amdahl
Establece el límite teórico de ganancia de velocidad (speedup) que se puede obtener al paralelizar una tarea.

$$S_{latencia}(s) = \frac{1}{(1-p) + \frac{p}{s}}$$

Donde:
*   $S$: Speedup total.
*   $p$: Proporción del programa que es paralelizable.
*   $s$: Número de procesadores (speedup de la parte paralela).

**Lección**: Si el 10% de tu programa es secuencial (no paralelizable), el speedup máximo nunca superará 10x, sin importar si usas 1 millón de procesadores.

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2017-2026.</p>